In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import timeit
from sklearn.preprocessing import MinMaxScaler
from plotly.offline import init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split

# Restaurants in SF - Collaborative Filtering Model with Surprise Package

Henrieke Baunack, 
Melvin Ang, 5 March 2020, 290X 

In [2]:
df = pd.read_csv("newRestaurants.csv")
df.head(10)
df

,User_Name,"Dragon Beaux, San Francisco","Seven Hills, San Francisco","Acquerello, San Francisco","Betty Lou's Seafood and Grill, San Francisco","Sweet Maple, San Francisco","Ristorante Franchino, San Francisco","Harborview Restaurant & Bar, San Francisco","Nob Hill Cafe, San Francisco","Piccolo Forno, San Francisco",...,"Burma Superstar Restaurant, San Francisco","L'ardoise Bistro, San Francisco","La Ciccia, San Francisco","New Thai Elephant, San Francisco","E Tutto Qua, San Francisco","Pacific Cafe, San Francisco","Kokkari Estiatorio, San Francisco","Eight Am, San Francisco","Jeanne D'Arc Restaurant, San Francisco","Pier Market Seafood Restaurant, San Francisco"
0,milkdemcows,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,Frank-in-SF,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chris F,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,oregonpoppa,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,yvonnechang2019,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20719,Lynn A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
20720,Ozarkatv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
20721,Hansen5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
20722,Gigi318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [3]:
df = df.drop_duplicates()
df

,User_Name,"Dragon Beaux, San Francisco","Seven Hills, San Francisco","Acquerello, San Francisco","Betty Lou's Seafood and Grill, San Francisco","Sweet Maple, San Francisco","Ristorante Franchino, San Francisco","Harborview Restaurant & Bar, San Francisco","Nob Hill Cafe, San Francisco","Piccolo Forno, San Francisco",...,"Burma Superstar Restaurant, San Francisco","L'ardoise Bistro, San Francisco","La Ciccia, San Francisco","New Thai Elephant, San Francisco","E Tutto Qua, San Francisco","Pacific Cafe, San Francisco","Kokkari Estiatorio, San Francisco","Eight Am, San Francisco","Jeanne D'Arc Restaurant, San Francisco","Pier Market Seafood Restaurant, San Francisco"
0,milkdemcows,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,Frank-in-SF,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Chris F,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,oregonpoppa,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,yvonnechang2019,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20719,Lynn A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
20720,Ozarkatv,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
20721,Hansen5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
20722,Gigi318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [4]:
#df = df.iloc[:, 0:30]
number_elements = df.size
number_nulls_nas = df.isna().sum().sum()
sparsity = number_nulls_nas/number_elements
sparsity

0.9437288634797747

In [10]:
sum(df.count(axis='columns',  numeric_only=True) > 1)

2039

In [8]:
df.drop(df[df.count(axis='columns',  numeric_only=True) < 2].index, inplace=True)
df

/Users/henriekebaunack/anaconda3/envs/data-x/lib/python3.7/site-packages/pandas/core/frame.py:4102: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,User_Name,"Dragon Beaux, San Francisco","Seven Hills, San Francisco","Acquerello, San Francisco","Betty Lou's Seafood and Grill, San Francisco","Sweet Maple, San Francisco","Ristorante Franchino, San Francisco","Harborview Restaurant & Bar, San Francisco","Nob Hill Cafe, San Francisco","Piccolo Forno, San Francisco",...,"Burma Superstar Restaurant, San Francisco","L'ardoise Bistro, San Francisco","La Ciccia, San Francisco","New Thai Elephant, San Francisco","E Tutto Qua, San Francisco","Pacific Cafe, San Francisco","Kokkari Estiatorio, San Francisco","Eight Am, San Francisco","Jeanne D'Arc Restaurant, San Francisco","Pier Market Seafood Restaurant, San Francisco"
0,milkdemcows,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,Chris F,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TravelingAuntMarian,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,UnrepentantFoodie,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,UnrepentantFoodie,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550,Ron C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0
18569,Erik L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0
18814,James W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0
18815,James W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0


In [11]:
df_cropped = df
display(df_cropped)

,User_Name,"Dragon Beaux, San Francisco","Seven Hills, San Francisco","Acquerello, San Francisco","Betty Lou's Seafood and Grill, San Francisco","Sweet Maple, San Francisco","Ristorante Franchino, San Francisco","Harborview Restaurant & Bar, San Francisco","Nob Hill Cafe, San Francisco","Piccolo Forno, San Francisco",...,"Burma Superstar Restaurant, San Francisco","L'ardoise Bistro, San Francisco","La Ciccia, San Francisco","New Thai Elephant, San Francisco","E Tutto Qua, San Francisco","Pacific Cafe, San Francisco","Kokkari Estiatorio, San Francisco","Eight Am, San Francisco","Jeanne D'Arc Restaurant, San Francisco","Pier Market Seafood Restaurant, San Francisco"
0,milkdemcows,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
2,Chris F,3.0,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,TravelingAuntMarian,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,UnrepentantFoodie,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,UnrepentantFoodie,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18550,Ron C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0
18569,Erik L,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0
18814,James W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,5.0
18815,James W,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0


In [14]:
sum(df_cropped.count(axis='columns',  numeric_only=True) > 5)

615

In [15]:
df_restaurants = df_cropped.columns[1:].array
df_restaurants

<PandasArray>
[              'Pacific Catch, San Francisco_x',
           'Liholiho Yacht Club, San Francisco',
                      'Frascati, San Francisco',
     'Taylor Street Coffee Shop, San Francisco',
         'Fino Bar & Ristorante, San Francisco',
    'Pazzia Ristorante Italiano, San Francisco',
        'El Nuevo Frutilandia, San Francisco_x',
                       'Trestle, San Francisco',
         'Restaurant Gary Danko, San Francisco',
             'Ristorante Milano, San Francisco',
                      'Za Pizza, San Francisco',
                         'Zazie, San Francisco',
          'Reed and Greenough, San Francisco_x',
                     'Surisan, San Francisco_x',
         'State Bird Provisions, San Francisco',
     "Brenda's French Soul Food, San Francisco",
 'Sotto Mare Oysteria & Seafood, San Francisco',
                       'Frances, San Francisco',
                         'SPIN San Francisco_x',
                          'Mymy, San Francisco',
      

In [12]:
#convert to Surprise package dataframe format

df_colnames = df_cropped.columns[1:].array
df_colnames
df2 = pd.melt(df_cropped, id_vars=['User_Name'], value_vars=df_colnames, var_name='Restaurant', value_name='Rating')
df2.shape
display(df2)

,User_Name,Restaurant,Rating
0,milkdemcows,"Dragon Beaux, San Francisco",4.0
1,Chris F,"Dragon Beaux, San Francisco",3.0
2,TravelingAuntMarian,"Dragon Beaux, San Francisco",5.0
3,UnrepentantFoodie,"Dragon Beaux, San Francisco",5.0
4,UnrepentantFoodie,"Dragon Beaux, San Francisco",2.0
...,...,...,...
75438,Ron C,"Pier Market Seafood Restaurant, San Francisco",5.0
75439,Erik L,"Pier Market Seafood Restaurant, San Francisco",5.0
75440,James W,"Pier Market Seafood Restaurant, San Francisco",5.0
75441,James W,"Pier Market Seafood Restaurant, San Francisco",5.0


In [13]:
df_new = df2.drop_duplicates()

In [14]:
df_new

,User_Name,Restaurant,Rating
0,milkdemcows,"Dragon Beaux, San Francisco",4.0
1,Chris F,"Dragon Beaux, San Francisco",3.0
2,TravelingAuntMarian,"Dragon Beaux, San Francisco",5.0
3,UnrepentantFoodie,"Dragon Beaux, San Francisco",5.0
4,UnrepentantFoodie,"Dragon Beaux, San Francisco",2.0
...,...,...,...
75437,Anthony D,"Pier Market Seafood Restaurant, San Francisco",4.0
75438,Ron C,"Pier Market Seafood Restaurant, San Francisco",5.0
75439,Erik L,"Pier Market Seafood Restaurant, San Francisco",5.0
75440,James W,"Pier Market Seafood Restaurant, San Francisco",5.0


In [27]:
"""#filter to reduce dataset 
min_place_ratings = 5
filter_place = df_new['Restaurant'].value_counts() > min_place_ratings
filter_place = filter_place[filter_place].index.tolist()
#df2.groupby('Place').count()['Rating'] 

min_user_ratings = 2
filter_users = df_new.groupby('User_Name').count()['Rating'] > min_user_ratings #min 3 ratings 
filter_users = filter_users[filter_users].index.tolist()
#df2.groupby('Name').count().sort_values(by=['Rating'], ascending=False)['Rating']

df_new2 = df_new[(df_new['Restaurant'].isin(filter_place)) & (df_new['User_Name'].isin(filter_users))]
print('The original data frame shape:\t{}'.format(df_new.shape))
print('The new data frame shape:\t{}'.format(df_new2.shape))"""

The original data frame shape:	(45493, 3)
The new data frame shape:	(32616, 3)


In [15]:
df_new = df_new.fillna(0)
df_new.shape

(72742, 3)

## divide all ratings by 10 so the scale becomes 1-5
#df_new["Rating"] = df_new["Rating"].div(10)
df_new2.describe()

In [16]:
df_new.describe()

,Rating
count,72742.000000
mean,0.287647
std,1.118557
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,5.000000


In [17]:
#initiate Surprise
reader = Reader(rating_scale=(0, 5))
Surprise_data = Dataset.load_from_df(df_new[['User_Name', 'Restaurant', 'Rating']], reader)

# Try Different Algorithms

In [18]:
#train on whole set and predict
# Retrieve the trainset.    
full_trainset = Surprise_data.build_full_trainset()

# Build an algorithm, and train it.
# Build an algorithm, and train it.
sim_options = {'name': 'cosine', # way to compute similarity matrix
               'user_based': True #Similarity based on similar users
               }
algoKNNBasic = KNNBasic(sim_options=sim_options)
algoKNNBasic.fit(full_trainset)


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [ ]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoSVD = SVD()
algoSVD.fit(full_trainset)

In [18]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsSVD = algoSVD.fit(trainset).test(testset)
accuracy.rmse(predictionsSVD)

RMSE: 0.0620


0.06202285794985639

#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNBaseline = KNNBaseline(sim_options=sim_options)
algoKNNBaseline.fit(full_trainset)

In [19]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNBasic = algoKNNBasic.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNBasic)

Computing the cosine similarity matrix...


/Users/henriekebaunack/anaconda3/envs/data-x/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning:

invalid value encountered in double_scalars



Done computing similarity matrix.
RMSE: 1.1365


1.1365341489390712

In [20]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNBaseline = algoKNNBaseline.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNBaseline)

NameError: name 'algoKNNBaseline' is not defined

In [154]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoBaselineOnly = BaselineOnly()
algoBaselineOnly.fit(full_trainset)

Estimating biases using als...


In [155]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsBaselineOnly = algoBaselineOnly.fit(trainset).test(testset)
accuracy.rmse(predictionsBaselineOnly)

Estimating biases using als...
RMSE: 1.4846


1.4845762391657442

In [157]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoCoClustering = CoClustering()
algoCoClustering.fit(full_trainset)

In [158]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsCoClustering = algoCoClustering.fit(trainset).test(testset)
accuracy.rmse(predictionsCoClustering)

RMSE: 1.5126


1.512581985164737

In [159]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoNormalPredictor = NormalPredictor()
algoNormalPredictor.fit(full_trainset)

In [160]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsNormalPredictor = algoNormalPredictor.fit(trainset).test(testset)
accuracy.rmse(predictionsNormalPredictor)

RMSE: 1.8636


1.8636400092448973

In [161]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNWithMeans = KNNWithMeans(sim_options=sim_options)
algoKNNWithMeans.fit(full_trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [162]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNWithMeans = algoKNNWithMeans.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNWithMeans)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4360


1.4360302776389788

In [163]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
sim_options = {'name': 'cosine',
               'user_based': True #Similarity based on similar users
               }
algoKNNWithZScore = KNNWithZScore(sim_options=sim_options)
algoKNNWithZScore.fit(full_trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [164]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsKNNWithZScore = algoKNNWithZScore.fit(trainset).test(testset)
accuracy.rmse(predictionsKNNWithZScore)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.4394


1.4393954529215478

In [166]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoSVDpp = SVDpp()
algoSVDpp.fit(full_trainset)

In [167]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsSVDpp = algoSVDpp.fit(trainset).test(testset)
accuracy.rmse(predictionsSVDpp)

RMSE: 1.4821


1.4820760563768778

In [173]:
#train on whole set and predict
# Retrieve the trainset.
full_trainset = Surprise_data.build_full_trainset()
algoSlopeOne = SlopeOne()
algoSlopeOne.fit(full_trainset)

In [174]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25, random_state=100)
#algo = BaselineOnly(bsl_options=bsl_options)
predictionsSlopeOne = algoSlopeOne.fit(trainset).test(testset)
accuracy.rmse(predictionsSlopeOne)

RMSE: 1.5009


1.5009175288343197

# Predicting

In [21]:
df_new

,User_Name,Restaurant,Rating
0,milkdemcows,"Dragon Beaux, San Francisco",4.0
1,Chris F,"Dragon Beaux, San Francisco",3.0
2,TravelingAuntMarian,"Dragon Beaux, San Francisco",5.0
3,UnrepentantFoodie,"Dragon Beaux, San Francisco",5.0
4,UnrepentantFoodie,"Dragon Beaux, San Francisco",2.0
...,...,...,...
75437,Anthony D,"Pier Market Seafood Restaurant, San Francisco",4.0
75438,Ron C,"Pier Market Seafood Restaurant, San Francisco",5.0
75439,Erik L,"Pier Market Seafood Restaurant, San Francisco",5.0
75440,James W,"Pier Market Seafood Restaurant, San Francisco",5.0


In [22]:
df_new.drop_duplicates()

,User_Name,Restaurant,Rating
0,milkdemcows,"Dragon Beaux, San Francisco",4.0
1,Chris F,"Dragon Beaux, San Francisco",3.0
2,TravelingAuntMarian,"Dragon Beaux, San Francisco",5.0
3,UnrepentantFoodie,"Dragon Beaux, San Francisco",5.0
4,UnrepentantFoodie,"Dragon Beaux, San Francisco",2.0
...,...,...,...
75437,Anthony D,"Pier Market Seafood Restaurant, San Francisco",4.0
75438,Ron C,"Pier Market Seafood Restaurant, San Francisco",5.0
75439,Erik L,"Pier Market Seafood Restaurant, San Francisco",5.0
75440,James W,"Pier Market Seafood Restaurant, San Francisco",5.0


In [40]:
# converting to strings does not help
#df_new2["User_Name"] = df_new2["User_Name"].to_string()
#df_new2["Restaurant"] = df_new2["Restaurant"].to_string()

In [25]:
#predict 
uid = "UnrepentantFoodie" # raw user id (as in the ratings file). They are **strings**!
iid = "Dragon Beaux, San Francisco" # raw item id (restrauran in this case)
# get a prediction for specific users and items.
pred=algoKNNBasic.predict(uid, iid, r_ui=5, verbose=True) # r_ui: if you know the original rating you can put it htere to compare with the estaimation



user: UnrepentantFoodie item: Dragon Beaux, San Francisco r_ui = 5.00   est = 0.86   {'actual_k': 10, 'was_impossible': False}


In [184]:
uid = str("834tamih")
user_list = []
place_list = []
rating_list = []
for i in range(len(df_placenames)):
    iid = df_placenames[i]
    #df_estimated_ratings = pd.DataFrame(algo.predict(uid, iid, verbose=True)[3])
    user_list.append(algoKNNBasic.predict(uid, iid, verbose=True)[0])
    place_list.append(algoKNNBasic.predict(uid, iid, verbose=True)[1])
    rating_list.append(algoKNNBasic.predict(uid, iid, verbose=True)[3])

user: 834tamih   item: 16_Avenue_Tiled_Steps r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: 16_Avenue_Tiled_Steps r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: 16_Avenue_Tiled_Steps r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: California_Academy_of_Sciences r_ui = None   est = 0.26   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: California_Academy_of_Sciences r_ui = None   est = 0.26   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: California_Academy_of_Sciences r_ui = None   est = 0.26   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Alcatraz_Island r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Alcatraz_Island r_ui = None   est = 0.00   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Alcatraz_Island r_ui = None   est = 0.00   {'actua

user: 834tamih   item: Fisherman_s_Wharf r_ui = None   est = 0.12   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Fisherman_s_Wharf r_ui = None   est = 0.12   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Japanese_Tea_Garden r_ui = None   est = 0.50   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Japanese_Tea_Garden r_ui = None   est = 0.50   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Japanese_Tea_Garden r_ui = None   est = 0.50   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Ghirardelli_Square r_ui = None   est = 0.09   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Ghirardelli_Square r_ui = None   est = 0.09   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Ghirardelli_Square r_ui = None   est = 0.09   {'actual_k': 40, 'was_impossible': False}
user: 834tamih   item: Chinatown  r_ui = None   est = 0.20   {'actual_k': 40, 'was_impossible': False}
user: 83

In [185]:
#Print final result table
df_estimate_result = pd.DataFrame(user_list, columns =['Name'])
df_estimate_result["Place"] = pd.DataFrame(place_list)
df_estimate_result = df_estimate_result.merge(df_new[df_new["Name"] == uid], on='Place', how='outer').drop(['Name_y'], axis=1)
df_estimate_result["Estimated Rating"] = pd.DataFrame(rating_list)
df_estimate_result = df_estimate_result.rename(columns={"Name_x": "Name"})
df_estimate_result = df_estimate_result.rename(columns={"Rating": "Original Rating"})
df_estimate_result["Absolute Error"] = abs(df_estimate_result["Original Rating"] - df_estimate_result["Estimated Rating"])
df_estimate_result.sort_values(by=['Estimated Rating'],ascending=False)

,Name,Place,Original Rating,Estimated Rating,Absolute Error
23,834tamih,Presidio_of_San_Francisco,5.0,3.956940,1.043060
24,834tamih,Baker_Beach,5.0,2.966907,2.033093
25,834tamih,Angel_Island_State_Park,5.0,2.175897,2.824103
20,834tamih,San_Francisco_Botanical_Garden,4.0,1.953024,2.046976
7,834tamih,Lands_End,0.0,0.927657,0.927657
4,834tamih,Exploratorium,0.0,0.543513,0.543513
33,834tamih,Haight_Ashbury,0.0,0.522273,0.522273
16,834tamih,Legion_of_Honor,0.0,0.517162,0.517162
28,834tamih,Japanese_Tea_Garden,0.0,0.503824,0.503824
14,834tamih,San_Francisco_Museum_of_Modern_Art_SFMOMA,0.0,0.496313,0.496313


In [71]:
#train and test set to check RMSE of model
trainset, testset = train_test_split(Surprise_data, test_size=0.25)
#algo = BaselineOnly(bsl_options=bsl_options)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Computing the cosine similarity matrix...


/Users/melvinang/anaconda3/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning:

invalid value encountered in double_scalars



Done computing similarity matrix.
RMSE: 1.4203


1.4203341765796793

In [72]:
df_result = pd.DataFrame(predictions)
df_result

,uid,iid,r_ui,est,details
0,kelsyf2016,San_Francisco_Museum_of_Modern_Art_SFMOMA,0.0,0.000000,"{'actual_k': 8, 'was_impossible': False}"
1,TandCAdventures,Exploratorium,0.0,0.582080,"{'actual_k': 24, 'was_impossible': False}"
2,traveler1327,Legion_of_Honor,0.0,0.724931,"{'actual_k': 40, 'was_impossible': False}"
3,anarayan,California_Academy_of_Sciences,0.0,0.931495,"{'actual_k': 16, 'was_impossible': False}"
4,CLT29,Ghirardelli_Square,0.0,0.625144,"{'actual_k': 34, 'was_impossible': False}"
5,Alok_Bahl,Exploratorium,5.0,0.435653,"{'actual_k': 8, 'was_impossible': False}"
6,thakersuresh,Exploratorium,0.0,0.091051,"{'actual_k': 16, 'was_impossible': False}"
7,Whatsinaname7172,San_Francisco_Botanical_Garden,0.0,0.000000,"{'actual_k': 35, 'was_impossible': False}"
8,LiannaS539,Ghirardelli_Square,0.0,0.967402,"{'actual_k': 40, 'was_impossible': False}"
9,Pickids,Twin_Peaks,0.0,0.618845,"{'actual_k': 39, 'was_impossible': False}"


In [80]:
#Check estimates for certain user
df_result[df_result["uid"] == "834tamih"]

,uid,iid,r_ui,est,details
3668,834tamih,16_Avenue_Tiled_Steps,0.0,0.674253,"{'actual_k': 19, 'was_impossible': False}"
8827,834tamih,Legion_of_Honor,0.0,0.000000,"{'actual_k': 2, 'was_impossible': False}"
9764,834tamih,Fisherman_s_Wharf,0.0,0.050817,"{'actual_k': 35, 'was_impossible': False}"
10500,834tamih,Palace_of_Fine_Arts_Theatre,0.0,0.000000,"{'actual_k': 2, 'was_impossible': False}"
12277,834tamih,De_Young_Museum,0.0,0.938937,"{'actual_k': 21, 'was_impossible': False}"
15972,834tamih,Presidio_of_San_Francisco,5.0,2.184342,"{'actual_k': 21, 'was_impossible': False}"
16222,834tamih,Twin_Peaks,0.0,1.037484,"{'actual_k': 22, 'was_impossible': False}"
17659,834tamih,Haight_Ashbury,0.0,0.974513,"{'actual_k': 10, 'was_impossible': False}"
26234,834tamih,Oracle_Park,0.0,0.000000,"{'actual_k': 7, 'was_impossible': False}"
29313,834tamih,Lands_End,0.0,4.500000,"{'actual_k': 2, 'was_impossible': False}"


In [77]:
#Check highest estimates
df_result.sort_values(by=['est'],ascending=False)

,uid,iid,r_ui,est,details
21560,ConnorYM,Lands_End,0.0,5.000000,"{'actual_k': 1, 'was_impossible': False}"
5958,K393YT,Golden_Gate_Park,0.0,5.000000,"{'actual_k': 2, 'was_impossible': False}"
23012,BethB246,or5-Golden_Gate_Bridge,0.0,5.000000,"{'actual_k': 1, 'was_impossible': False}"
24259,AzraelMel,Golden_Gate_Park,0.0,5.000000,"{'actual_k': 2, 'was_impossible': False}"
29313,834tamih,Lands_End,0.0,4.500000,"{'actual_k': 2, 'was_impossible': False}"
18287,537beverleyg,or5-Golden_Gate_Bridge,0.0,4.042536,"{'actual_k': 27, 'was_impossible': False}"
9495,Laurie48,Lands_End,5.0,4.000000,"{'actual_k': 1, 'was_impossible': False}"
4819,Adelaide_girl_SA,Palace_of_Fine_Arts_Theatre,5.0,4.000000,"{'actual_k': 1, 'was_impossible': False}"
12086,Seasondtraveller,De_Young_Museum,5.0,3.836621,"{'actual_k': 4, 'was_impossible': False}"
560,joanneb648,or5-Golden_Gate_Bridge,4.0,3.833333,"{'actual_k': 36, 'was_impossible': False}"


In [90]:
#Compile original user rating and estimated rating by model
dfresult_combined = df_new[df_new["Name"] == "834tamih"]
dfresult_combined = dfresult_combined.rename(columns={"Place": "iid"})
dfresult_combined = dfresult_combined.merge(df_result[df_result["uid"] == "834tamih"], on='iid', how='outer').drop(['uid', 'r_ui', 'details'], axis=1)
dfresult_combined

,Name,iid,Rating,est
0,834tamih,16_Avenue_Tiled_Steps,0.0,0.674253
1,834tamih,California_Academy_of_Sciences,0.0,NaN
2,834tamih,Alcatraz_Island,0.0,0.078566
3,834tamih,Cable_Cars,0.0,0.050157
4,834tamih,Exploratorium,0.0,NaN
5,834tamih,or5-Golden_Gate_Bridge,0.0,NaN
6,834tamih,Golden_Gate_Park,0.0,1.057313
7,834tamih,Lands_End,0.0,4.500000
8,834tamih,Oracle_Park,0.0,0.000000
9,834tamih,Palace_of_Fine_Arts_Theatre,0.0,0.000000


In [ ]:
predictions2 = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)